In [1]:
import lib.PrimeGen as pg

In [2]:
import math
import gmpy2
import random

In [5]:
class RSA:
    def __init__(self, bits):
        pgen = pg.PrimeGen()
        p = pgen.Generate(bits) # Generate a *safe prime* with __bits__ BITS.
        print ('Computed Random Prime p.')
        q = pgen.Generate(bits) # Generate a *safe prime* with __bits__ BITS.
        print ('Computed Random Prime q.')
        phi = (p-1)*(q-1)       # euler's phi function for n
        
        self.n = p * q
        self.e = 2**16 + 1
        self.__d = gmpy2.invert(self.e, phi)
        print ('RSA-%d Initiated Successfully.' %(bits))
    
    def __padhexa(s, l):
        return '0x' + s[2:].zfill(l)
        
    def __GenerateByteSequence(byte_length):
        hex_arr = list(map(str,\
                       list(range(1, 10)) + ['A', 'B', 'C', 'D', 'E', 'F']))
        secure_random = random.SystemRandom()
        tmp = byte_length * 8
        res = ''
        while (tmp is not 0):
            word = secure_random.choice(hex_arr)
            size = 4
            if (size <= tmp):
                tmp -= size
                res += word
        return res
        
    def __PadMessage(plainText, n):
        EB = math.ceil(n.bit_length()/8) # byte-size of Encryption Block (EB)
        DB = math.ceil(plainText.bit_length()/8)       # byte-size of Data Block (DB)
        PS = EB - 3 - DB                        # byte-size of Padding String (PS)
        if not(DB <= EB - 11):
            raise ValueError('Message > Encryption Block - 11.')
        else:
            PaddingString = GenerateByteSequence(PS)
        paddedText = '0x0002'+PaddingString+'00'+hex(plainText)[2:]
        return paddedText
        
    def __PublicKey(self):
        return self.e, self.n
    
    def __PrivateKey(self):
        return self.__d, self.n
    
    def Encrypt(self, plainText, padding=False):
        [e, n] = self.__PublicKey()
        if padding:
            plainTextPadded = self.__PadMessage(plainText, n)
            tmp = pow(int(plainTextPadded,16), e, n)
        else:
            tmp = pow(plainText, e, n)
        return tmp
    
    def Decrypt(self, cipherText, padding=False):
        [d, n] = self.__PrivateKey()
        
        decipherText = pow(cipherText, d, n)
        
        if padding:
            byte_difference = abs(len(hex(rsa256.n)) - len(hex(decipherText)))
            if (byte_difference >= 3):
                bytelen = len(hex(decipherText)[2:])
                decipherText = self.__padhexa(hex(decipherText), bytelen + 3)
            
            if (decipherText[2:6] == '0002'):
                decipherText = int(tmp4.split('00')[-1], 16)
            else:
                decipherText = ''
            
        return decipherText

In [6]:
rsa256 = RSA(256)

Computed Random Prime p.
Computed Random Prime q.
RSA-256 Initiated Successfully.


In [7]:
m1 = random.randrange(1024)
c1 = rsa256.Encrypt(m1)
mm1 = rsa256.Decrypt(c1)

print ('Initial Message: ', m1, '\n')
print ('Encrypted Message: ', c1, '\n')
print ('Decrypted Message: ', mm1, '\n')

Initial Message:  393 

Encrypted Message:  588210979185370403140197108073432297609208703505031697734029373881391784097146187354103851290372698909011528304103649204876992020255947309114652953490662 

Decrypted Message:  393 



In [8]:
m2 = random.randrange(1024)
c2 = rsa256.Encrypt(m2)
mm2 = rsa256.Decrypt(c2)

print ('Initial Message: ', m2, '\n')
print ('Encrypted Message: ', c2, '\n')
print ('Decrypted Message: ', mm2, '\n')

# Show Homomorphic Properties
print (rsa256.Decrypt(c1 * c2) == (m1 * m2) % rsa256.n)

Initial Message:  866 

Encrypted Message:  2923963189550801750218817639073003891448694154484835805710699644191382625452354441503008525243708727271222740222639918579285314415282371852255634349732994 

Decrypted Message:  866 

True


In [9]:
rsa256.n

4323484774871250219024594876710528792250684426671328209655598649387816479224171030180961975187746377045536518276206716663819655852483296766958848370064749

# Make The PKCS #1 v1.5 Padded Cipher Text

In [14]:
EB = math.ceil(rsa256.n.bit_length()/8) # byte-size of Encryption Block (EB)
DB = math.ceil(m1.bit_length()/8)       # byte-size of Data Block (DB)
PS = EB - 3 - DB                        # byte-size of Padding String (PS)

print ('[Target] CipherText Length: ', EB)
print ('\nMessage: ', hex(m1))
print ('\nMessage Length: ', DB)
print ('\nPadding Length: ', PS)


if not(DB <= EB - 11):
    print('Can\'t Encrypt This.')
    sys.exit(0)
else:
    PaddingString = GenerateByteSequence(PS)

print ('\nComputed Padding: ', PaddingString)
print ('\nComputed Padding Length: ', math.ceil(len(bin(int(PaddingString, 16))[2:])/8))

cipher_padded = '0x0002'+PaddingString+'00'+hex(m1)[2:]

print ('\nPadded CipherText: ', cipher_padded)
print ('\nPadded CipherText Length:\nbytelen(0x0002) (=2) + bytelen(PaddingString) (=59) + bytelen(0x00) (=1) + bytelen(message) (=2) = ',\
       2 + math.ceil(len(bin(int(PaddingString, 16))[2:])/8) + 1 + math.ceil(len(bin(m1)[2:])/8))


cipher_encrypted = pow(int(cipher_padded,16), rsa256.e, rsa256.n)
print ('\nEncrypted CipherText: ', cipher_encrypted)

[Target] CipherText Length:  64

Message:  0x189

Message Length:  2

Padding Length:  59

Computed Padding:  C4777E73DD49A1DE644A3C66A1BA1D85ACE8615EA1111627BAD54DE3D3268BF71B7DF527191455A6D9AFFC3E389DC638DB21FDCCBFCFB1C12AD7BA

Computed Padding Length:  59

Padded CipherText:  0x0002C4777E73DD49A1DE644A3C66A1BA1D85ACE8615EA1111627BAD54DE3D3268BF71B7DF527191455A6D9AFFC3E389DC638DB21FDCCBFCFB1C12AD7BA00189

Padded CipherText Length:
bytelen(0x0002) (=2) + bytelen(PaddingString) (=59) + bytelen(0x00) (=1) + bytelen(message) (=2) =  64

Encrypted CipherText:  3892676952579687901993713734381467029478506585601326194897693584381022391076430602507063348142606209242412187253537005359325748221532993980671694669169754


In [15]:
plain_1 = rsa256.Decrypt(cipher_encrypted)

In [17]:
# print (EB)
# print (math.ceil(plain.bit_length()/8))
# print ('0x%0*x' % (len(hex(rsa256.n)[2:]), plain))

if (len(hex(plain_1)) < len(hex(rsa256.n))):
    tmp4 = padhexa(hex(plain_1), len(hex(plain_1)[2:]) + 3)

int(tmp4.split('00')[2], 16)


393

In [18]:
EB = math.ceil(rsa256.n.bit_length()/8) # byte-size of Encryption Block (EB)
DB = math.ceil(m2.bit_length()/8)       # byte-size of Data Block (DB)
PS = EB - 3 - DB                        # byte-size of Padding String (PS)

print ('[Target] CipherText Length: ', EB)
print ('\nMessage: ', hex(m2))
print ('\nMessage Length: ', DB)
print ('\nPadding Length: ', PS)


if not(DB <= EB - 11):
    print('Can\'t Encrypt This.')
    sys.exit(0)
else:
    PaddingString = GenerateByteSequence(PS)

print ('\nComputed Padding: ', PaddingString)
print ('\nComputed Padding Length: ', math.ceil(len(bin(int(PaddingString, 16))[2:])/8))

cipher_padded = '0x0002'+PaddingString+'00'+hex(m2)[2:]

print ('\nPadded CipherText: ', cipher_padded)
print ('\nPadded CipherText Length:\nbytelen(0x0002) (=2) + bytelen(PaddingString) (=60) + bytelen(0x00) (=1) + bytelen(message) (=1) = ',\
       2 + math.ceil(len(bin(int(PaddingString, 16))[2:])/8) + 1 + math.ceil(len(bin(m2)[2:])/8))


cipher_encrypted_2 = pow(int(cipher_padded,16), rsa256.e, rsa256.n)
print ('\nEncrypted CipherText: ', cipher_encrypted_2)

[Target] CipherText Length:  64

Message:  0x362

Message Length:  2

Padding Length:  59

Computed Padding:  F43ABD4AF53B4DCFB8B3174BB49A5DEA25944B72A519F2D4A3E7AAEADFE7D2E8D7A117DD9B3CB7FA535CAFD8BE8644915FDF64511A9ED5AC1156D9

Computed Padding Length:  59

Padded CipherText:  0x0002F43ABD4AF53B4DCFB8B3174BB49A5DEA25944B72A519F2D4A3E7AAEADFE7D2E8D7A117DD9B3CB7FA535CAFD8BE8644915FDF64511A9ED5AC1156D900362

Padded CipherText Length:
bytelen(0x0002) (=2) + bytelen(PaddingString) (=60) + bytelen(0x00) (=1) + bytelen(message) (=1) =  64

Encrypted CipherText:  3102166782669551284215508552002243022823180267192622323751686012530241182121001869866637565342253514507997659482150754413987442387103710304740646821677168


In [19]:
plain_2 = rsa256.Decrypt(cipher_encrypted_2)

In [21]:
hex(plain_2)

'0x2f43abd4af53b4dcfb8b3174bb49a5dea25944b72a519f2d4a3e7aaeadfe7d2e8d7a117dd9b3cb7fa535cafd8be8644915fdf64511a9ed5ac1156d900362'

In [45]:
print(len(hex(plain_2)))
print(len(hex(rsa256.n)))


if (abs(len(hex(rsa256.n)) - len(hex(plain_2))) >= 3):
    tmp4 = padhexa(hex(plain_2), len(hex(plain_2)[2:]) + 3)

    
print(tmp4[2:6])
    
int(tmp4.split('00')[-1], 16)

126
130
0002


866

In [12]:
def padhexa(s, l):
    return '0x' + s[2:].zfill(l)

In [11]:
def GenerateByteSequence(byte_length):
    hex_arr = list(map(str,\
                   list(range(1, 10)) + ['A', 'B', 'C', 'D', 'E', 'F']))
    
    secure_random = random.SystemRandom()
    tmp = byte_length * 8
    res = ''
    while (tmp is not 0):
        word = secure_random.choice(hex_arr)
        size = 4
        if (size <= tmp):
            tmp -= size
            res += word
    return res